In [1]:


import cv2
import os
import socket
import sys



steeringWheelHost = "raspberrypi.mshome.net"
steeringWheelPort = "5000"
DLCarHUDHost = "mattpi.mshome.net"
DLCarHUDPort = "4001"

from datetime import date
import datetime

from time import sleep

date = date.today()
time = datetime.datetime.now().time()
timeString = str(time)
dateString = str(date)
captureID = 'capture' + dateString[0:4] + dateString[5:7] + dateString[8:10] + timeString[0:2] + timeString[3:5]
print(captureID)

pictureDirectory = captureID + "/Pictures/"

if not os.path.exists(captureID):
    os.makedirs(captureID)
    
if not os.path.exists(pictureDirectory):
    os.makedirs(pictureDirectory)
    


# For sending request to retrieve accelerometer coordinates
def netcat(host, port, content):
    data = ""
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((host, int(port)))
    s.sendall(content.encode())
    s.shutdown(socket.SHUT_WR)
    while True:
        data = s.recv(8192)
        if not data:
            break
        data = data.decode("utf-8") 
        data = data.rstrip(' \0')
        return str(data)
    s.close()

camera_port = 1
if len(sys.argv) > 1:
    print(str(sys.argv))
    #camera_port = int(sys.argv[1])

#Number of frames to throw away while the camera adjusts to light levels
ramp_frames = 40000

# Now we can initialize the camera capture object with the cv2.VideoCapture class.
# All it needs is the index to a camera port.
try:
    camera = cv2.VideoCapture(camera_port)
except Warning:
   print("Failure")


global imageNum
imageNum = 0
# Captures a single image from the camera and returns it in PIL format

def get_image():
    global imageNum
    # read is the easiest way to get a full image out of a VideoCapture object.
    imageCaptureSuccessful = 0
    imageWriteSuccessful = 0
    try:
        retval, im = camera.read()
    except:
        print("Image capture failure")
        return ""
    try:
        file = "./" + str(captureID) + "/Pictures/"+str(captureID)+"image" + str(imageNum)+ ".png"
        imageNum = imageNum + 1
        cv2.imwrite(file, im)
    except:
        print("Image write failure")
        return ""
    try:
        steeringInfo = netcat(steeringWheelHost,steeringWheelPort, "request")
    except:
        print("Obtaining Steering Wheel Data Failure")
        try:
            os.remove(file)
        except:
            print("File removal failed")
        return ""
    try:
        dlCarHUDInfo = netcat(DLCarHUDHost,DLCarHUDPort, "request")
    except:
        print("Obtaining DLCarHUD Data Failure")
        try:
            os.remove(file)
        except:
            print("File removal failed")
        os.remove(file)
        return ""
    segments = dlCarHUDInfo.split(',')
    speed = segments[1]
    speed = (speed.split(':'))[1]
    speed = float(speed)
    if(speed < -1.0):
        print("Speed insufficient")
        try:
            os.remove(file)
        except:
            print("File removal failed")
        sleep(0.2)
        return ""
        
    dataFile = captureID + "/accelClass.txt"
    time = datetime.datetime.now().time()
    f = open(dataFile, 'a')
    f.write(dlCarHUDInfo)
    f.write(',')
    f.write(str(time))
    f.write(',')
    f.write(str(file))
    f.write(',')
    f.write(steeringInfo)  # python will convert \n to os.linesep
    f.write("\n")  # python will convert \n to os.linesep
    f.close()  # you can omit in most cases as the destructor will call it
    print(steeringInfo)
    print(dlCarHUDInfo)
    sleep(0.35)
    
    print(time)
    return im

for i in range(ramp_frames):
    temp = get_image()

# You'll want to release the camera, otherwise you won't be able to create a new
# capture object until your script exits
del(camera)



capture201708060146
['c:\\users\\user\\appdata\\local\\programs\\python\\python36-32\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\User\\AppData\\Roaming\\jupyter\\runtime\\kernel-d368dbb8-5be4-443d-9cfc-c415aed747ab.json']
Roll: 99.218647, Pitch: -5.743290, Yaw: -101.823736, MX: 0.100830, MY: 0.986279
RPM: 0.00, Speed: 0.0, Throttle: 0.0, IntakeTemp: 0, EngineCoolantTemp: 0, ManifoldABS: 0, Latitude: 47.619541, Longitude: -122.198008, TrackingAngle: 33.600000

01:46:41.045567
Roll: 99.175972, Pitch: -5.687640, Yaw: -101.690909, MX: 0.093604, MY: 0.986475
RPM: 0.00, Speed: 0.0, Throttle: 0.0, IntakeTemp: 0, EngineCoolantTemp: 0, ManifoldABS: 0, Latitude: 47.619541, Longitude: -122.198008, TrackingAngle: 33.600000

01:46:41.497312
Roll: 99.553395, Pitch: -4.677828, Yaw: -100.709719, MX: 0.087061, MY: 0.986768
RPM: 0.00, Speed: 0.3, Throttle: 0.0, IntakeTemp: 0, EngineCoolantTemp: 0, ManifoldABS: 0, Latitude: 47.619542, Longitude: -122.198009, TrackingAngle: 33.600000

0

01:46:55.082501
Roll: 99.949471, Pitch: -3.585414, Yaw: -100.061206, MX: 0.063135, MY: 0.987109
RPM: 0.00, Speed: 0.0, Throttle: 0.0, IntakeTemp: 0, EngineCoolantTemp: 0, ManifoldABS: 0, Latitude: 47.619525, Longitude: -122.198006, TrackingAngle: 33.600000

01:46:55.510252
Roll: 99.943624, Pitch: -3.582629, Yaw: -99.641928, MX: 0.062988, MY: 0.987012
RPM: 0.00, Speed: 0.1, Throttle: 0.0, IntakeTemp: 0, EngineCoolantTemp: 0, ManifoldABS: 0, Latitude: 47.619523, Longitude: -122.198005, TrackingAngle: 33.600000

01:46:55.956650
Roll: 99.935114, Pitch: -3.579113, Yaw: -99.273029, MX: 0.063086, MY: 0.986328
RPM: 0.00, Speed: 0.1, Throttle: 0.0, IntakeTemp: 0, EngineCoolantTemp: 0, ManifoldABS: 0, Latitude: 47.619523, Longitude: -122.198005, TrackingAngle: 33.600000

01:46:56.424065
Roll: 99.929431, Pitch: -3.587034, Yaw: -99.160263, MX: 0.062891, MY: 0.986328
RPM: 0.00, Speed: 0.1, Throttle: 0.0, IntakeTemp: 0, EngineCoolantTemp: 0, ManifoldABS: 0, Latitude: 47.619522, Longitude: -122.19800

01:47:10.708018
Roll: 100.005956, Pitch: -3.468839, Yaw: -99.554208, MX: 0.060889, MY: 0.986328
RPM: 0.00, Speed: 0.1, Throttle: 0.0, IntakeTemp: 0, EngineCoolantTemp: 0, ManifoldABS: 0, Latitude: 47.619498, Longitude: -122.198009, TrackingAngle: 33.600000

01:47:11.166909
Roll: 99.945004, Pitch: -3.276043, Yaw: -99.630911, MX: 0.061035, MY: 0.986621
RPM: 0.00, Speed: 0.1, Throttle: 0.0, IntakeTemp: 0, EngineCoolantTemp: 0, ManifoldABS: 0, Latitude: 47.619498, Longitude: -122.198009, TrackingAngle: 33.600000

01:47:11.621906
Roll: 99.985118, Pitch: -3.360559, Yaw: -99.199871, MX: 0.060889, MY: 0.986523
RPM: 0.00, Speed: 0.1, Throttle: 0.0, IntakeTemp: 0, EngineCoolantTemp: 0, ManifoldABS: 0, Latitude: 47.619498, Longitude: -122.198009, TrackingAngle: 33.600000

01:47:12.097650
Roll: 100.013053, Pitch: -3.407202, Yaw: -99.263542, MX: 0.060645, MY: 0.986133
RPM: 0.00, Speed: 0.1, Throttle: 0.0, IntakeTemp: 0, EngineCoolantTemp: 0, ManifoldABS: 0, Latitude: 47.619498, Longitude: -122.1980

01:47:25.647326
Roll: 100.036740, Pitch: -3.358385, Yaw: -99.479602, MX: 0.058691, MY: 0.986719
RPM: 0.00, Speed: 0.0, Throttle: 0.0, IntakeTemp: 0, EngineCoolantTemp: 0, ManifoldABS: 0, Latitude: 47.619505, Longitude: -122.198014, TrackingAngle: 33.600000

01:47:26.114786
Roll: 100.055462, Pitch: -3.355939, Yaw: -99.821835, MX: 0.058691, MY: 0.986523
RPM: 0.00, Speed: 0.0, Throttle: 0.0, IntakeTemp: 0, EngineCoolantTemp: 0, ManifoldABS: 0, Latitude: 47.619505, Longitude: -122.198014, TrackingAngle: 33.600000

01:47:26.594352
Roll: 100.043406, Pitch: -3.359940, Yaw: -100.328785, MX: 0.058740, MY: 0.985840
RPM: 0.00, Speed: 0.1, Throttle: 0.0, IntakeTemp: 0, EngineCoolantTemp: 0, ManifoldABS: 0, Latitude: 47.619505, Longitude: -122.198014, TrackingAngle: 33.600000

01:47:27.067135
Roll: 100.057784, Pitch: -3.365229, Yaw: -99.621362, MX: 0.058594, MY: 0.986035
RPM: 0.00, Speed: 0.1, Throttle: 0.0, IntakeTemp: 0, EngineCoolantTemp: 0, ManifoldABS: 0, Latitude: 47.619505, Longitude: -122.1

KeyboardInterrupt: 

In [ ]:
import cv2
import os
import socket
import sys
import time
steeringWheelHost = "192.168.137.168"
steeringWheelPort = "5000"
# For sending request to retrieve accelerometer coordinates
def netcat(host, port, content):
    data = ""
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((host, int(port)))
    s.sendall(content.encode())
    s.shutdown(socket.SHUT_WR)
    while True:
        data = s.recv(8192)
        if not data:
            break
        data = data.decode("utf-8") 
        data = data.rstrip(' \0')
        return str(data)
    s.close()
while(1):
    steeringInfo = netcat(steeringWheelHost,steeringWheelPort, "request") 
    print(steeringInfo)
    time.sleep(0.1)


In [ ]:
dlCarHUDInfo = netcat(DLCarHUDHost,DLCarHUDPort, "request")
print(dlCarHUDInfo)